In [4]:
import yaml

with open("privacy_config.yaml", "r") as file:
    privacy_config = yaml.safe_load(file)

In [5]:
import hashlib

def hash_value(value, algo='sha256'):
    if pd.isna(value):
        return value
    h = hashlib.new(algo)
    h.update(str(value).encode())
    return h.hexdigest()

def anonymize_dataframe(df, config):
    if config["anonymization"]["hash_identifiers"]:
        fields = config["anonymization"]["fields_to_hash"]
        algo = config["anonymization"]["hash_algorithm"]
        for field in fields:
            if field in df.columns:
                df[field] = df[field].apply(lambda x: hash_value(x, algo))
    return df


In [17]:
def has_access(user_role, config):
    if not config["access_control"]["role_based"]:
        return True
    return user_role in config["access_control"]["allowed_roles"]



    if not has_access(user_role, privacy_config):
        raise PermissionError("User does not have access to this dataset.")

In [9]:
from cryptography.fernet import Fernet

# Generate key (should be securely stored, not hardcoded in production)
key = Fernet.generate_key()
fernet = Fernet(key)

def encrypt_column(data, key):
    return data.apply(lambda x: fernet.encrypt(str(x).encode()).decode())

def decrypt_column(data, key):
    return data.apply(lambda x: fernet.decrypt(str(x).encode()).decode())

# Example usage
# df["sensitive_column"] = encrypt_column(df["sensitive_column"], key)


In [10]:
import logging

log_level = privacy_config["logging"]["level"].upper()
logging.basicConfig(level=log_level)

if not privacy_config["logging"]["log_sensitive_data"]:
    logging.info("Sensitive data logging disabled.")


INFO:root:Sensitive data logging disabled.


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
df = pd.read_csv('features_del.csv')
df = anonymize_dataframe(df, privacy_config)
X = df.drop(columns=["label"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
user_role = "data_scientist"
if has_access(user_role, privacy_config):
    print("Training...")
    model = GradientBoostingClassifier(random_state=42)
    model.fit(X_train, y_train)

# 5. Evaluation
    y_pred = model.predict(X_test)


Training...
